In [ ]:
# Dataset configuration (resize images, load images, split dataset into train, validation, test)

import os
import cv2
import numpy as np

img_size = (150, 150)

train_dir = '../dataset/train'
validation_dir = '../dataset/val'
test_dir = '../dataset/test'

def resize_images_from_directory(directory):
    images = []
    labels = []
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        for filename in os.listdir(label_dir):
            img_path = os.path.join(label_dir, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, img_size)
            images.append(img)
            labels.append(label)

    if (directory == train_dir):
        images = images[:3500]
        labels = labels[:3500]

    return np.array(images), np.array(labels)


X_train, y_train = resize_images_from_directory(train_dir)
X_validation, y_validation = resize_images_from_directory(validation_dir)
X_test, y_test = resize_images_from_directory(test_dir)

X_train = X_train.reshape(X_train.shape[0], -1)
X_validation = X_validation.reshape(X_validation.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

In [ ]:
# Preprocessing pictures

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_validation = scaler.transform(X_validation)
X_test = scaler.transform(X_test)

In [ ]:
# Generate synthetic samples using SMOTE

import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from collections import Counter

def count_files(folder_path):
    count = 0
    for _, _, files in os.walk(folder_path):
        count += len(files)
    return count

folder_path_normal = "../dataset/train/NORMAL"
folder_path_pneumonia = "../dataset/train/PNEUMONIA"
num_files_normal = count_files(folder_path_normal)
num_files_pneumonia = count_files(folder_path_pneumonia)
print("Number of files in the folder normal :", num_files_normal)
print("Number of files in the folder pneumonia :", num_files_pneumonia)

counter_before = Counter(y_train)
print("Class Distribution Before SMOTE:", counter_before)

smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)  

counter_after = Counter(y_train)
print("Class Distribution After SMOTE:", counter_after)

plt.figure(figsize=(10, 5))
plt.bar(counter_before.keys(), counter_before.values(), color='b', label='Before SMOTE')
plt.bar(counter_after.keys(), counter_after.values(), color='r', alpha=0.5, label='After SMOTE')
plt.xlabel('Class')
plt.ylabel('Number of Instances')
plt.title('Class Distribution Before and After SMOTE')
plt.legend()
plt.show()

In [ ]:
# Search for the best hyperparameters using GridSearchCV

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.svm import LinearSVC

param = {
    'C':[0.001, 0.01, 0.1, 1, 10, 100],
    'penalty': ['l1', 'l2'],
    'loss': ['hinge', 'squared_hinge'],
    'dual': [True, False],
}

grid = GridSearchCV(LinearSVC(), param, cv=5, scoring='accuracy')
grid.fit(X_validation, y_validation)

best_params = grid.best_params_
best_score = grid.best_score_
best_estimator = grid.best_estimator_

print("Best parameters:", best_params)
print("Best score:", best_score)
print('Score with best estimator:', best_estimator.score(X_test, y_test))

confusion_matrix(y_validation, best_estimator.predict(X_validation))

In [ ]:
# Train the model and score it

model = LinearSVC(C=0.01, dual=False, loss='squared_hinge', penalty='l1')
model.fit(X_train, y_train)
model.score(X_validation, y_validation)

In [ ]:
# Use pipeline to scale the data and train the model

# from sklearn.pipeline import make_pipeline

# model = make_pipeline(StandardScaler(), LinearSVC(C=0.01, dual=False, loss='squared_hinge', penalty='l1'))
# params = {
# }
# grid = GridSearchCV(model, params, cv=5, scoring='accuracy')
# grid.fit(X_train, y_train)
# grid.score(X_test, y_test)

# predictions = grid.predict(X_test)

# num_correct = sum(y_test == predictions)
# total_predictions = len(y_test)

# print(num_correct / total_predictions)

In [ ]:
# Evaluate the model

predictions = model.predict(X_test)
indices = np.arange(len(y_test))

def success_rate() :
    num_correct = sum(y_test == predictions)
    total_predictions = len(y_test)
    return num_correct / total_predictions

def success_rate_per_class(class_name):
    expected = y_test[y_test == class_name]
    predicted = predictions[y_test == class_name]
    num_correct = sum(expected == predicted)
    total_predictions = len(expected)
    return num_correct / total_predictions

print("Success Rate:", success_rate())
print("Success Rate for Pneumonia:", success_rate_per_class('PNEUMONIA'))
print("Success Rate for Normal:", success_rate_per_class('NORMAL'))

plt.figure(figsize=(10, 6))
plt.plot(indices, y_test, label='Expected', marker='o')
plt.plot(indices, predictions, label='Predicted', marker='x')
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Expected vs. Predicted Results')
plt.legend()
plt.grid(True)
plt.show()